In [20]:
import keras

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

import numpy as np
import pandas as pd
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Concatenate, Dropout
from keras.models import Model

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2061633207998210902
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 18437397696836759472
physical_device_desc: "device: XLA_CPU device"
]


In [13]:
from ast import literal_eval
data_train = pd.read_pickle('drive/My Drive/Colab Notebooks/fakenews-master/data/news_proc_12_3_19.pickle')
data_train = data_train[['type', 'text', 'title']]
data_train['type'].unique()

array(['real', 'fake'], dtype=object)

In [0]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [0]:
def clean_str(string):
    """
    Cleaning of dataset
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [16]:
from ast import literal_eval
data_train = pd.read_pickle('drive/My Drive/Colab Notebooks/fakenews-master/data/news_proc_12_3_19.pickle')
data_train = data_train[['type', 'text', 'title']]
data_train['type'].unique()

array(['real', 'fake'], dtype=object)

In [0]:
data_train['type'] = data_train['type'].replace('fake',1)
data_train['type'] = data_train['type'].replace('real',0)

In [0]:
texts = []
labels = []

for i in range(data_train.text.shape[0]):
    text1 = data_train.title[i]
    text2 = data_train.text[i]
    text = str(text1) +""+ str(text2)
    texts.append(text)
    labels.append(data_train.type[i])

In [21]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 43449 unique tokens.


In [22]:
# Pad input sequences
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(labels),num_classes = 2)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)


Shape of data tensor: (19155, 1000)
Shape of label tensor: (19155, 2)


In [23]:
# Train test validation Split
from sklearn.model_selection import train_test_split
print(type(data))
indices = np.arange(len(data))
print(indices)
np.random.shuffle(indices)
l = []
d = []
for each in indices:
  d.append(data[each])
  l.append(labels[each])
data =np.asarray(d)
labels = np.asarray(l)
print(type(data[0]))
x_train, x_test, y_train, y_test = train_test_split( data, labels, test_size=0.20, random_state=42)
x_test, x_val, y_test, y_val = train_test_split( data, labels, test_size=0.50, random_state=42)
print('Size of train, validation, test:', len(y_train), len(y_val), len(y_test))




<class 'numpy.ndarray'>
[    0     1     2 ... 19152 19153 19154]
<class 'numpy.ndarray'>
Size of train, validation, test: 15324 9578 9577


In [24]:
GLOVE_DIR = "drive/My Drive/Colab Notebooks/fakenews-master/data/"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
params_lstm = {
    'lr': 0.2639850638741692,
    'momentum': 0.08255873365806747,
    'dropout': 0.10591561319817788,
    'dense1': 73,
    'dense2': 24,
    'lstmTitle': 46,
    'lstmContent': 231
}

In [36]:
from tensorflow import keras as k
#input_title
input = k.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), name='input')
inp = k.layers.Embedding(output_dim=EMBEDDING_DIM, input_dim=43450, 
                         weights=[embedding_matrix], trainable=False)(input)
x = k.layers.Bidirectional(k.layers.LSTM(params_lstm['lstmTitle']))(inp)
#CudnnLSTM
#x = k.layers.Bidirectional(k.layers.CuDNNLSTM(params_lstm['lstmTitle']))(inp)

# #input_content
# content_input = k.layers.Input(shape=(len_content,), name='content_input')
# inp2 = k.layers.Embedding(output_dim=300, input_dim=50000, 
#                          weights=[embed_mat], trainable=False)(content_input)
# x2 = k.layers.Bidirectional(k.layers.LSTM(params_lstm['lstmContent']))(inp2)
# #CudnnLSTM
# #x2 = k.layers.Bidirectional(k.layers.CuDNNLSTM(params_lstm['lstmContent']))(inp2)

# #Merge
# x = k.layers.concatenate([x, x2])


#Common part
x = k.layers.Dense(params_lstm['dense1'], activation='relu')(x)
x = k.layers.Dropout(params_lstm['dropout'])(x)
x = k.layers.Dense(params_lstm['dense2'], activation='relu')(x)
x = k.layers.Dropout(0.2)(x)

out = k.layers.Dense(2, activation='sigmoid')(x)

#Build model
model = k.models.Model(inputs=[input], outputs=[out])

#model.compile(k.optimizers.SGD(lr=params_lstm['lr'], 
#                               momentum=params_lstm['momentum']), 
#                               loss='categorical_crossentropy', metrics=['acc'])

model.compile(tf.train.MomentumOptimizer(learning_rate=params_lstm['lr'], 
                                         momentum=params_lstm['momentum']), 
                                         loss='binary_crossentropy', metrics=['acc'])

model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 1000)]            0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 1000, 100)         4345000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 92)                54096     
_________________________________________________________________
dense_9 (Dense)              (None, 73)                6789      
_________________________________________________________________
dropout_6 (Dropout)          (None, 73)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 24)                1776      
_________________________________________________________________
dropout_7 (Dropout)          (None, 24)                0   

In [37]:
 model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=3, batch_size=128)

Train on 15324 samples, validate on 9578 samples
Epoch 1/3
15324/15324 [==============================] - 408s 27ms/sample - loss: 0.6831 - acc: 0.5750 - val_loss: 0.6828 - val_acc: 0.5776
Epoch 2/3
15324/15324 [==============================] - 418s 27ms/sample - loss: 0.6819 - acc: 0.5776 - val_loss: 0.6811 - val_acc: 0.5777
Epoch 3/3
15324/15324 [==============================] - 418s 27ms/sample - loss: 0.6815 - acc: 0.5776 - val_loss: 0.6810 - val_acc: 0.5777
